In [2]:
!cat german_credit_data | head -n 10
!curl ...

   1   6   4  12   5   5   3   4   1  67   3   2   1   2   1   0   0   1   0   0   1   0   0   1   1 
   2  48   2  60   1   3   2   2   1  22   3   1   1   1   1   0   0   1   0   0   1   0   0   1   2 
   4  12   4  21   1   4   3   3   1  49   3   1   2   1   1   0   0   1   0   0   1   0   1   0   1 
   1  42   2  79   1   4   3   4   2  45   3   1   2   1   1   0   0   0   0   0   0   0   0   1   1 
   1  24   3  49   1   3   3   4   4  53   3   2   2   1   1   1   0   1   0   0   0   0   0   1   2 
   4  36   2  91   5   3   3   4   4  35   3   1   2   2   1   0   0   1   0   0   0   0   1   0   1 
   4  24   2  28   3   5   3   4   2  53   3   1   1   1   1   0   0   1   0   0   1   0   0   1   1 
   2  36   2  69   1   3   3   2   3  35   3   1   1   2   1   0   1   1   0   1   0   0   0   0   1 
   4  12   2  31   4   4   1   4   1  61   3   1   1   1   1   0   0   1   0   0   1   0   1   0   1 
   2  30   4  52   1   1   4   2   3  28   3   2   1   1   1   1   0   1   0   0  

In [37]:
import pandas as pd

In [42]:
data = pd.read_csv("german_credit_data", header=None, sep="\\s+")
data = data.as_matrix()

In [51]:
X = data[:, :24]
Y = data[:, 24]

In [71]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

model = DecisionTreeClassifier()
print cross_val_score(model, X, y, cv=5).mean()

0.675


## Writing linear classificator

In [148]:
import numpy as np
from math import log
from math import exp

from scipy.optimize import minimize

class Logistic(object):
    
    def __init__(self, gamma=0.01, reg='l1'):
        self.gamma = gamma
        self.reg = reg
        self.w = None
        
    def get_reg(self, weights):
        if self.reg == 'l1':
            return sum(abs(weights))
        elif self.reg == 'l2':
            return sum(weights ** 2)
        
    def get_confidence(self, x, w):
        return np.dot(w[:-1], x) + w[-1] 
    
    def predict_one(self, x):
        return 1 if self.get_confidence(x, self.w) > 0 else -1
    
    def predict(self, X_test):
        return np.array([self.predict_one(x) for x in X_test])
    
    def get_loss(self, margin):
        #print margin
        return np.log(1.0 + np.exp(-margin))
    
    def get_risk(self, X_train, y_train, w):
        risk = 0
        for x, y in zip(X_train, y_train):
            margin = y * self.get_confidence(x, w)
            loss = self.get_loss(margin)
            risk += loss
            
        risk += self.gamma * self.get_reg(w[:-1])
        return risk
    
    def get_min_func(self, X_train, y_train):
        return lambda w: self.get_risk(X_train, y_train, w)
    
    def fit(self, X_train, y_train):
        min_func = self.get_min_func(X_train, y_train)
        dim = len(X_train[0]) + 1
        min_obj = minimize(min_func, np.array([1.0] * dim))
        #print min_obj
        w = min_obj.x
        self.w = w
        print self.w
        

In [149]:
log_model = Logistic()

Y_norm = (Y - 1.5) * 2

log_model.fit(X, Y_norm)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.]


In [150]:
preds = log_model.predict(X)

In [151]:
print preds != Y

[False  True False False  True False False False False  True  True  True
 False  True False  True False False  True False False False False False
 False False False False False  True False False False False False  True
 False  True False False False False False False  True False False False
 False False False False False False  True False  True False False  True
 False False  True  True False False False False  True False False False
 False False  True False  True False False False  True False False False
 False False False  True False  True False False  True False False  True
 False False False False False False False False False  True  True False
 False False False False False  True False False  True False  True False
  True False False False  True False False  True False  True False  True
 False False False False False  True False False False False False  True
 False False False False False False False False False False False  True
 False False False False False False False False Fa